# Building a Dataset and then Using it in Stata

Let's construct a Dataset for a Gravity Model Regression

In [1]:
import pandas as pd

## Data

The data can be found in the `data` directory and is contained in two files

1. `data/exports_2013.csv` -- Data From Comtrade
2. `data/cepii_2013.csv` -- Date from Cepii

These datasets have been shrunk to just 1-year due to size of files. 

But this exercise can be expanded into a proper panel setting with multiple years

When working with data that has the potential to be moderately large it is always a good idea to bring in just a few rows initially to see what the data file contains. 

This allows you to review the contents and select the data that you actually need through `column` selectors etc.

In [2]:
exports = pd.read_csv("data/exports_2013.csv", nrows=10)

In [3]:
exports

,year,iiso3c,eiso3c,value
0,2013,ABW,BEL,774353.0
1,2013,ABW,BHS,4712537.0
2,2013,ABW,CHE,17812626.0
3,2013,ABW,CHN,25319168.0
4,2013,ABW,COL,22160086.0
5,2013,ABW,DOM,9316793.0
6,2013,ABW,ESP,8708237.0
7,2013,ABW,FRA,13262797.0
8,2013,ABW,GBR,88157843.0
9,2013,ABW,JAM,2428953.0


We now bring in all the `trade` data from the `csv` file as the data isn't too large to work with and we want to use all the columns

In [4]:
exports = pd.read_csv("data/exports_2013.csv")

In [5]:
exports

,year,iiso3c,eiso3c,value
0,2013,ABW,BEL,774353.0
1,2013,ABW,BHS,4712537.0
2,2013,ABW,CHE,17812626.0
3,2013,ABW,CHN,25319168.0
4,2013,ABW,COL,22160086.0
...,...,...,...,...
22690,2013,IRQ,JOR,NaN
22691,2013,IRQ,LBN,NaN
22692,2013,IRQ,LBY,NaN
22693,2013,IRQ,SYR,NaN


**Cepii** provides a dataset for `gravity` models that includes country-pairs with data relating to those countries as:
1. country pair properties such as distance between them
2. and `country` properties such as `gdp` in a given year

In this case the source file just contains `2013` data

In [6]:
cepii = pd.read_csv("data/cepii_2013.csv")

In [7]:
cepii

,year,iso3_o,iso3_d,iso3num_o,iso3num_d,contig,comlang_off,comcol,dist,distcap,distw,distwces,pop_o,pop_o.1,gdp_o,gdp_d,gdpcap_o,gdpcap_d
0,2013,ABW,ABW,533.0,533.0,0.0,0.0,0.0,5.225,5.225,25.094,23.047,102.911,102.911,NaN,NaN,NaN,NaN
1,2013,ABW,AFG,533.0,4.0,0.0,0.0,0.0,13257.814,13257.814,13168.224,13166.367,102.911,102.911,NaN,2.030967e+07,NaN,0.665
2,2013,ABW,AGO,533.0,24.0,0.0,0.0,0.0,9516.913,9516.913,9587.316,9584.193,102.911,102.911,NaN,1.241782e+08,NaN,5.783
3,2013,ABW,AIA,533.0,660.0,0.0,0.0,0.0,983.268,983.268,976.897,976.892,102.911,102.911,NaN,NaN,NaN,NaN
4,2013,ABW,ALB,533.0,8.0,0.0,0.0,0.0,9091.742,9091.742,9091.576,9091.466,102.911,102.911,NaN,1.292324e+07,NaN,4.659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61499,2013,ZWE,YMD,716.0,720.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14149.648,14149.648,1.349000e+07,NaN,0.953,NaN
61500,2013,ZWE,YUG,716.0,890.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14149.648,14149.648,1.349000e+07,NaN,0.953,NaN
61501,2013,ZWE,ZAF,716.0,710.0,1.0,1.0,0.0,2186.206,926.174,1258.552,1101.438,14149.648,14149.648,1.349000e+07,3.506301e+08,0.953,6.618
61502,2013,ZWE,ZMB,716.0,894.0,1.0,1.0,1.0,396.804,396.804,583.795,525.073,14149.648,14149.648,1.349000e+07,2.682087e+07,0.953,1.845


In [8]:
cepii.columns

Index(['year', 'iso3_o', 'iso3_d', 'iso3num_o', 'iso3num_d', 'contig',
       'comlang_off', 'comcol', 'dist', 'distcap', 'distw', 'distwces',
       'pop_o', 'pop_o.1', 'gdp_o', 'gdp_d', 'gdpcap_o', 'gdpcap_d'],
      dtype='object')

Let's see what columns the exports data contains

In [9]:
exports.columns

Index(['year', 'iiso3c', 'eiso3c', 'value'], dtype='object')

Now we could merge this information together indexed by `year, iiso3c, eiso3c` (for exports) and `year, iso3_d, iso3_o` (for cepii)

In [10]:
exports.merge(cepii, how='left', left_on=['year', 'iiso3c', 'eiso3c'], right_on=['year', 'iso3_d', 'iso3_o'])

,year,iiso3c,eiso3c,value,iso3_o,iso3_d,iso3num_o,iso3num_d,contig,comlang_off,...,dist,distcap,distw,distwces,pop_o,pop_o.1,gdp_o,gdp_d,gdpcap_o,gdpcap_d
0,2013,ABW,BEL,774353.0,BEL,ABW,56.0,533.0,0.0,1.0,...,7847.070,7847.070,7843.255,7843.006,11195.138,11195.138,5.248055e+08,NaN,46.878,NaN
1,2013,ABW,BHS,4712537.0,BHS,ABW,44.0,533.0,0.0,0.0,...,1588.515,1588.515,1634.515,1628.143,377.374,377.374,8.420000e+06,NaN,22.312,NaN
2,2013,ABW,CHE,17812626.0,CHE,ABW,756.0,533.0,0.0,0.0,...,8056.332,8056.332,8074.210,8073.511,8081.482,8081.482,6.854342e+08,NaN,84.815,NaN
3,2013,ABW,CHN,25319168.0,CHN,ABW,156.0,533.0,0.0,0.0,...,14155.351,14155.351,14590.924,14560.277,1357380.005,1357380.005,9.240271e+09,NaN,6.807,NaN
4,2013,ABW,COL,22160086.0,COL,ABW,170.0,533.0,0.0,1.0,...,1036.634,1036.634,929.589,861.245,48321.404,48321.404,3.784153e+08,NaN,7.831,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22690,2013,IRQ,JOR,NaN,JOR,IRQ,400.0,368.0,1.0,1.0,...,811.116,811.116,872.717,859.304,6459.000,6459.000,3.367850e+07,2.293273e+08,5.214,6.862
22691,2013,IRQ,LBN,NaN,LBN,IRQ,422.0,368.0,0.0,1.0,...,830.012,830.012,875.264,857.289,4467.390,4467.390,4.435242e+07,2.293273e+08,9.928,6.862
22692,2013,IRQ,LBY,NaN,LBY,IRQ,434.0,368.0,0.0,1.0,...,2911.806,2911.806,2706.425,2650.742,6201.521,6201.521,7.419953e+07,2.293273e+08,11.965,6.862
22693,2013,IRQ,SYR,NaN,SYR,IRQ,760.0,368.0,1.0,1.0,...,754.075,754.075,761.694,707.888,22845.551,22845.551,NaN,2.293273e+08,NaN,6.862


But one simplification strategy is to choose a common set of `index` column variables to make `merge` operations easier to read

In [11]:
exports.columns = ['year', 'iso3_d','iso3_o', 'evalue']

In [12]:
exports

,year,iso3_d,iso3_o,evalue
0,2013,ABW,BEL,774353.0
1,2013,ABW,BHS,4712537.0
2,2013,ABW,CHE,17812626.0
3,2013,ABW,CHN,25319168.0
4,2013,ABW,COL,22160086.0
...,...,...,...,...
22690,2013,IRQ,JOR,NaN
22691,2013,IRQ,LBN,NaN
22692,2013,IRQ,LBY,NaN
22693,2013,IRQ,SYR,NaN


Reorganising the columns to be in `origin` then `destination` order

In [13]:
dataset = exports[['year', 'iso3_o', 'iso3_d', 'evalue']]

In [14]:
dataset = dataset.merge(cepii, how='left', on=['year', 'iso3_o', 'iso3_d'])

In [15]:
dataset

,year,iso3_o,iso3_d,evalue,iso3num_o,iso3num_d,contig,comlang_off,comcol,dist,distcap,distw,distwces,pop_o,pop_o.1,gdp_o,gdp_d,gdpcap_o,gdpcap_d
0,2013,BEL,ABW,774353.0,56.0,533.0,0.0,1.0,0.0,7847.070,7847.070,7843.255,7843.006,11195.138,11195.138,5.248055e+08,NaN,46.878,NaN
1,2013,BHS,ABW,4712537.0,44.0,533.0,0.0,0.0,0.0,1588.515,1588.515,1634.515,1628.143,377.374,377.374,8.420000e+06,NaN,22.312,NaN
2,2013,CHE,ABW,17812626.0,756.0,533.0,0.0,0.0,0.0,8056.332,8056.332,8074.210,8073.511,8081.482,8081.482,6.854342e+08,NaN,84.815,NaN
3,2013,CHN,ABW,25319168.0,156.0,533.0,0.0,0.0,0.0,14155.351,14155.351,14590.924,14560.277,1357380.005,1357380.005,9.240271e+09,NaN,6.807,NaN
4,2013,COL,ABW,22160086.0,170.0,533.0,0.0,1.0,0.0,1036.634,1036.634,929.589,861.245,48321.404,48321.404,3.784153e+08,NaN,7.831,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22690,2013,JOR,IRQ,NaN,400.0,368.0,1.0,1.0,0.0,811.116,811.116,872.717,859.304,6459.000,6459.000,3.367850e+07,2.293273e+08,5.214,6.862
22691,2013,LBN,IRQ,NaN,422.0,368.0,0.0,1.0,0.0,830.012,830.012,875.264,857.289,4467.390,4467.390,4.435242e+07,2.293273e+08,9.928,6.862
22692,2013,LBY,IRQ,NaN,434.0,368.0,0.0,1.0,0.0,2911.806,2911.806,2706.425,2650.742,6201.521,6201.521,7.419953e+07,2.293273e+08,11.965,6.862
22693,2013,SYR,IRQ,NaN,760.0,368.0,1.0,1.0,0.0,754.075,754.075,761.694,707.888,22845.551,22845.551,NaN,2.293273e+08,NaN,6.862


Now we may want to select some of the information into a smaller dataframe containing data we intend to use for a regression

In [16]:
regdata = dataset[['year','iso3_o','iso3_d','evalue','dist','gdp_o', 'gdp_d']]

In [17]:
regdata

,year,iso3_o,iso3_d,evalue,dist,gdp_o,gdp_d
0,2013,BEL,ABW,774353.0,7847.070,5.248055e+08,NaN
1,2013,BHS,ABW,4712537.0,1588.515,8.420000e+06,NaN
2,2013,CHE,ABW,17812626.0,8056.332,6.854342e+08,NaN
3,2013,CHN,ABW,25319168.0,14155.351,9.240271e+09,NaN
4,2013,COL,ABW,22160086.0,1036.634,3.784153e+08,NaN
...,...,...,...,...,...,...,...
22690,2013,JOR,IRQ,NaN,811.116,3.367850e+07,2.293273e+08
22691,2013,LBN,IRQ,NaN,830.012,4.435242e+07,2.293273e+08
22692,2013,LBY,IRQ,NaN,2911.806,7.419953e+07,2.293273e+08
22693,2013,SYR,IRQ,NaN,754.075,NaN,2.293273e+08


## Comparing Cepii GDP Data to WDI Data

In [18]:
import wbgapi as wb

In [19]:
wb.series.info(q='gdp')

id,value
EG.GDP.PUSE.KO.PP,GDP per unit of energy use (PPP $ per kg of oil equivalent)
EG.GDP.PUSE.KO.PP.KD,GDP per unit of energy use (constant 2017 PPP $ per kg of oil equivalent)
EG.USE.COMM.GD.PP.KD,"Energy use (kg of oil equivalent) per $1,000 GDP (constant 2017 PPP)"
NY.GDP.DEFL.KD.ZG,"Inflation, GDP deflator (annual %)"
NY.GDP.DEFL.KD.ZG.AD,"Inflation, GDP deflator: linked series (annual %)"
NY.GDP.DEFL.ZS,GDP deflator (base year varies by country)
NY.GDP.DEFL.ZS.AD,GDP deflator: linked series (base year varies by country)
NY.GDP.DISC.CN,Discrepancy in expenditure estimate of GDP (current LCU)
NY.GDP.DISC.KN,Discrepancy in expenditure estimate of GDP (constant LCU)
NY.GDP.MKTP.CD,GDP (current US$)


In [20]:
wdi_gdp = wb.data.DataFrame(['NY.GDP.MKTP.CD'], time="YR2013")

In [21]:
wdi_gdp

,NY.GDP.MKTP.CD
economy,
ABW,2.701676e+09
AFG,2.056107e+10
AGO,1.367099e+11
ALB,1.277622e+10
AND,3.193704e+09
...,...
XKX,7.074778e+09
YEM,4.041524e+10
ZAF,3.668294e+11


In [22]:
regdata

,year,iso3_o,iso3_d,evalue,dist,gdp_o,gdp_d
0,2013,BEL,ABW,774353.0,7847.070,5.248055e+08,NaN
1,2013,BHS,ABW,4712537.0,1588.515,8.420000e+06,NaN
2,2013,CHE,ABW,17812626.0,8056.332,6.854342e+08,NaN
3,2013,CHN,ABW,25319168.0,14155.351,9.240271e+09,NaN
4,2013,COL,ABW,22160086.0,1036.634,3.784153e+08,NaN
...,...,...,...,...,...,...,...
22690,2013,JOR,IRQ,NaN,811.116,3.367850e+07,2.293273e+08
22691,2013,LBN,IRQ,NaN,830.012,4.435242e+07,2.293273e+08
22692,2013,LBY,IRQ,NaN,2911.806,7.419953e+07,2.293273e+08
22693,2013,SYR,IRQ,NaN,754.075,NaN,2.293273e+08


In [23]:
regdata.merge(wdi_gdp, how='left', left_on=['iso3_o'], right_index=True)

,year,iso3_o,iso3_d,evalue,dist,gdp_o,gdp_d,NY.GDP.MKTP.CD
0,2013,BEL,ABW,774353.0,7847.070,5.248055e+08,NaN,5.216427e+11
1,2013,BHS,ABW,4712537.0,1588.515,8.420000e+06,NaN,1.056840e+10
2,2013,CHE,ABW,17812626.0,8056.332,6.854342e+08,NaN,6.885042e+11
3,2013,CHN,ABW,25319168.0,14155.351,9.240271e+09,NaN,9.570406e+12
4,2013,COL,ABW,22160086.0,1036.634,3.784153e+08,NaN,3.821161e+11
...,...,...,...,...,...,...,...,...
22690,2013,JOR,IRQ,NaN,811.116,3.367850e+07,2.293273e+08,3.445444e+10
22691,2013,LBN,IRQ,NaN,830.012,4.435242e+07,2.293273e+08,4.690934e+10
22692,2013,LBY,IRQ,NaN,2911.806,7.419953e+07,2.293273e+08,6.550287e+10
22693,2013,SYR,IRQ,NaN,754.075,NaN,2.293273e+08,NaN


In [24]:
regdata = regdata.merge(wdi_gdp, how='left', left_on=['iso3_o'], right_index=True)
regdata = regdata.rename(columns={'NY.GDP.MKTP.CD' : 'wdi_gdp_o'})

In [25]:
regdata

,year,iso3_o,iso3_d,evalue,dist,gdp_o,gdp_d,wdi_gdp_o
0,2013,BEL,ABW,774353.0,7847.070,5.248055e+08,NaN,5.216427e+11
1,2013,BHS,ABW,4712537.0,1588.515,8.420000e+06,NaN,1.056840e+10
2,2013,CHE,ABW,17812626.0,8056.332,6.854342e+08,NaN,6.885042e+11
3,2013,CHN,ABW,25319168.0,14155.351,9.240271e+09,NaN,9.570406e+12
4,2013,COL,ABW,22160086.0,1036.634,3.784153e+08,NaN,3.821161e+11
...,...,...,...,...,...,...,...,...
22690,2013,JOR,IRQ,NaN,811.116,3.367850e+07,2.293273e+08,3.445444e+10
22691,2013,LBN,IRQ,NaN,830.012,4.435242e+07,2.293273e+08,4.690934e+10
22692,2013,LBY,IRQ,NaN,2911.806,7.419953e+07,2.293273e+08,6.550287e+10
22693,2013,SYR,IRQ,NaN,754.075,NaN,2.293273e+08,NaN


In [26]:
regdata['wdi_gdp_o'] = regdata['wdi_gdp_o'] / 1000

In [27]:
regdata = regdata.merge(wdi_gdp, how='left', left_on=['iso3_d'], right_index=True)
regdata = regdata.rename(columns={'NY.GDP.MKTP.CD' : 'wdi_gdp_d'})
regdata['wdi_gdp_d'] = regdata['wdi_gdp_d'] / 1000

In [28]:
regdata

,year,iso3_o,iso3_d,evalue,dist,gdp_o,gdp_d,wdi_gdp_o,wdi_gdp_d
0,2013,BEL,ABW,774353.0,7847.070,5.248055e+08,NaN,5.216427e+08,2.701676e+06
1,2013,BHS,ABW,4712537.0,1588.515,8.420000e+06,NaN,1.056840e+07,2.701676e+06
2,2013,CHE,ABW,17812626.0,8056.332,6.854342e+08,NaN,6.885042e+08,2.701676e+06
3,2013,CHN,ABW,25319168.0,14155.351,9.240271e+09,NaN,9.570406e+09,2.701676e+06
4,2013,COL,ABW,22160086.0,1036.634,3.784153e+08,NaN,3.821161e+08,2.701676e+06
...,...,...,...,...,...,...,...,...,...
22690,2013,JOR,IRQ,NaN,811.116,3.367850e+07,2.293273e+08,3.445444e+07,2.346484e+08
22691,2013,LBN,IRQ,NaN,830.012,4.435242e+07,2.293273e+08,4.690934e+07,2.346484e+08
22692,2013,LBY,IRQ,NaN,2911.806,7.419953e+07,2.293273e+08,6.550287e+07,2.346484e+08
22693,2013,SYR,IRQ,NaN,754.075,NaN,2.293273e+08,NaN,2.346484e+08


# Exporting dataset to Stata

In [29]:
regdata.to_stata("./data/gravity_model.dta", write_index=False)

----

# Running Gravity Model in Python

In [30]:
from numpy import log
import statsmodels.formula.api as smf

In [31]:
formula = "log(evalue) ~ log(gdp_o) + log(gdp_d) + log(dist)"
model = smf.ols(formula, regdata)
result = model.fit(cov_type='HC1')

In [32]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:            log(evalue)   R-squared:                       0.639
Model:                            OLS   Adj. R-squared:                  0.639
Method:                 Least Squares   F-statistic:                 1.175e+04
Date:                Tue, 30 Mar 2021   Prob (F-statistic):               0.00
Time:                        15:52:02   Log-Likelihood:                -49204.
No. Observations:               20334   AIC:                         9.842e+04
Df Residuals:                   20330   BIC:                         9.845e+04
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -14.5333      0.324    -44.876      0.0

We could use `ipystata` to run a regression at this point

In [33]:
import ipystata

IPyStata is loaded in batch mode.


In [34]:
%%stata -d regdata
gen levalue = ln(evalue)
gen lgdp_o = ln(gdp_o)
gen lgdp_d = ln(gdp_d)
gen ldist = ln(dist)
reg levalue lgdp_o lgdp_d ldist


(1,282 missing values generated)
(771 missing values generated)
(412 missing values generated)

      Source |       SS           df       MS      Number of obs   =    20,334
-------------+----------------------------------   F(3, 20330)     =  11997.99
       Model |  266473.973         3  88824.6575   Prob > F        =    0.0000
    Residual |  150508.973    20,330  7.40329429   R-squared       =    0.6391
-------------+----------------------------------   Adj R-squared   =    0.6390
       Total |  416982.945    20,333  20.5076942   Root MSE        =    2.7209

------------------------------------------------------------------------------
     levalue |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
      lgdp_o |   1.266673   .0083833   151.10   0.000     1.250241    1.283105
      lgdp_d |   1.025185    .008746   117.22   0.000     1.008043    1.042328
       ldist |  -1.395945   .0233

Perhaps you now want to add **exporter** dummies

In [35]:
%%stata -d regdata

#Exporter Dummies
encode iso3_o, g(iso3o)
gen levalue = ln(evalue)
gen lgdp_o = ln(gdp_o)
gen lgdp_d = ln(gdp_d)
gen ldist = ln(dist)
reg levalue lgdp_o lgdp_d ldist i.iso3o


Unknown #command(1,282 missing values generated)
(771 missing values generated)
(412 missing values generated)
note: 194.iso3o omitted because of collinearity

      Source |       SS           df       MS      Number of obs   =    20,334
-------------+----------------------------------   F(187, 20146)   =    276.19
       Model |  299971.916       187   1604.1279   Prob > F        =    0.0000
    Residual |  117011.029    20,146  5.80815194   R-squared       =    0.7194
-------------+----------------------------------   Adj R-squared   =    0.7168
       Total |  416982.945    20,333  20.5076942   Root MSE        =      2.41

------------------------------------------------------------------------------
     levalue |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
      lgdp_o |  -5.042138   .7786165    -6.48   0.000     -6.56829   -3.515986
      lgdp_d |   1.069977   .0077948   137.27   

Perhaps we want to replicate this in `python` we can compute these dummies in `pandas` as well

In [36]:
regdata = regdata.join(pd.get_dummies(regdata.iso3_o))

In [37]:
regdata

,year,iso3_o,iso3_d,evalue,dist,gdp_o,gdp_d,wdi_gdp_o,wdi_gdp_d,ABW,...,UZB,VCT,VEN,VNM,VUT,WSM,YEM,ZAF,ZMB,ZWE
0,2013,BEL,ABW,774353.0,7847.070,5.248055e+08,NaN,5.216427e+08,2.701676e+06,0,...,0,0,0,0,0,0,0,0,0,0
1,2013,BHS,ABW,4712537.0,1588.515,8.420000e+06,NaN,1.056840e+07,2.701676e+06,0,...,0,0,0,0,0,0,0,0,0,0
2,2013,CHE,ABW,17812626.0,8056.332,6.854342e+08,NaN,6.885042e+08,2.701676e+06,0,...,0,0,0,0,0,0,0,0,0,0
3,2013,CHN,ABW,25319168.0,14155.351,9.240271e+09,NaN,9.570406e+09,2.701676e+06,0,...,0,0,0,0,0,0,0,0,0,0
4,2013,COL,ABW,22160086.0,1036.634,3.784153e+08,NaN,3.821161e+08,2.701676e+06,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22690,2013,JOR,IRQ,NaN,811.116,3.367850e+07,2.293273e+08,3.445444e+07,2.346484e+08,0,...,0,0,0,0,0,0,0,0,0,0
22691,2013,LBN,IRQ,NaN,830.012,4.435242e+07,2.293273e+08,4.690934e+07,2.346484e+08,0,...,0,0,0,0,0,0,0,0,0,0
22692,2013,LBY,IRQ,NaN,2911.806,7.419953e+07,2.293273e+08,6.550287e+07,2.346484e+08,0,...,0,0,0,0,0,0,0,0,0,0
22693,2013,SYR,IRQ,NaN,754.075,NaN,2.293273e+08,NaN,2.346484e+08,0,...,0,0,0,0,0,0,0,0,0,0


Let's check the computation of dummies

In [38]:
regdata[['iso3_o', 'BEL']]

,iso3_o,BEL
0,BEL,1
1,BHS,0
2,CHE,0
3,CHN,0
4,COL,0
...,...,...
22690,JOR,0
22691,LBN,0
22692,LBY,0
22693,SYR,0


We can then join those columns into a string that can be used as part of the `statsmodels` formula

In [39]:
countries = " + ".join(regdata.columns[9:-2])  #Leaving off ZWE

In [40]:
countries

'ABW + AFG + AGO + ALB + ARE + ARG + ARM + ATG + AUS + AUT + AZE + BDI + BEL + BEN + BFA + BGD + BGR + BHR + BHS + BIH + BLR + BLZ + BMU + BOL + BRA + BRB + BRN + BTN + BWA + CAF + CAN + CHE + CHL + CHN + CIV + CMR + COG + COL + COM + CPV + CRI + CUB + CYM + CYP + CZE + DEU + DJI + DMA + DNK + DOM + DZA + ECU + EGY + ERI + ESP + EST + ETH + FIN + FJI + FRA + FRO + FSM + GAB + GBR + GEO + GHA + GIN + GMB + GNB + GNQ + GRC + GRD + GRL + GTM + GUY + HKG + HND + HRV + HTI + HUN + IDN + IND + IRL + IRN + IRQ + ISL + ISR + ITA + JAM + JOR + JPN + KAZ + KEN + KGZ + KHM + KIR + KNA + KOR + KWT + LAO + LBN + LBR + LBY + LCA + LKA + LSO + LTU + LUX + LVA + MAC + MAR + MDA + MDG + MDV + MEX + MHL + MKD + MLI + MLT + MMR + MNG + MNP + MOZ + MRT + MUS + MWI + MYS + NAM + NCL + NER + NGA + NIC + NLD + NOR + NPL + NZL + OMN + PAK + PAN + PER + PHL + PLW + PNG + POL + PRK + PRT + PRY + PYF + QAT + RUS + RWA + SAU + SDN + SEN + SGP + SLB + SLE + SLV + SMR + SOM + STP + SUR + SVK + SVN + SWE + SWZ + SYC

In [41]:
formula = "log(evalue) ~ log(gdp_o) + log(gdp_d) + log(dist) + " + countries
model = smf.ols(formula, regdata)
result = model.fit()

In [42]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:            log(evalue)   R-squared:                       0.719
Model:                            OLS   Adj. R-squared:                  0.717
Method:                 Least Squares   F-statistic:                     276.2
Date:                Tue, 30 Mar 2021   Prob (F-statistic):               0.00
Time:                        15:52:07   Log-Likelihood:                -46645.
No. Observations:               20334   AIC:                         9.367e+04
Df Residuals:                   20146   BIC:                         9.515e+04
Df Model:                         187                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     22.7282      8.035      2.829      0.0